# 🔥 Hotspot — More‑Like‑Chatroom

This notebook explains the **`hotspot--more-like-chatroom`** project from GitHub in an interactive, intermediate‑level way.

It walks through the architecture, key files, how messages flow, and includes runnable examples that let you experiment with the core ideas (sockets, simple server, and clients).

_This is not an exact copy of the repository; it is a guided, runnable explanation you can open, read and execute in a Jupyter environment._

## 📚 What this notebook covers
- Project overview & goals
- Network model (server, clients, hotspot idea)
- Walkthrough of the main server and client components
- Interactive cells to run a tiny local server and connect clients (on your machine)
- Tips for running on phones / devices and debugging

Run cells in order. If you run the server cell, keep that cell running — open a new terminal or notebook tab to run clients that connect to it.

## 🧭 Quick summary of the original GitHub project
- Purpose: create a chatroom that works over a local hotspot (no internet required). One device acts like a hotspot (server) and others connect as clients.
- Core ideas used: Python sockets (TCP), basic threading to handle multiple clients, message broadcasting, and simple text‑based protocol (e.g., exchange nickname first).
- Typical files in the repo: a server script, a client script, maybe helpers for message formatting and README with run instructions.

We will recreate and simplify those pieces here so you can run and test locally.

## ⚠️ Before you run anything
1. **Local machine only:** The interactive server/client examples in this notebook run on your machine (localhost) or your LAN IP. If you want to test across devices, run the server on the hotspot device and run clients on other devices that join the hotspot network.
2. **Ports & firewall:** Use a high port (like `5555`) and ensure your firewall allows connections.
3. **Run order:** Start the server cell first. Then run clients in separate terminals or notebook cells.
4. **Do not run multiple blocking server cells in the same notebook kernel** — if you need multiple shells, open terminals or new notebooks.


## 🏗 Architecture — How messages flow (short)
1. Client connects to Server (TCP socket)
2. Client sends a `NICKNAME` message to identify itself
3. Server stores client socket and nickname, then **broadcasts** messages it receives from any client to all other clients
4. When client disconnects, server removes it and broadcasts a `left` message

This is the classic single‑room chatroom model.

## 🧩 Minimal Server (runnable)
The cell below launches a small threaded chat server you can test locally. **Run this cell** to start the server. It will block in the notebook until you stop it (Ctrl+C in a terminal) — in notebooks this may not be ideal; run it in a separate terminal if you prefer.

If you want to run it in the notebook, run the cell and then open a new notebook or terminal to run clients.

In [ ]:
import socket
import threading
import sys

class TinyChatServer:
    def __init__(self, host='0.0.0.0', port=5555):
        self.host = host
        self.port = port
        self.server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.clients = []  # list of (socket, nickname)

    def broadcast(self, msg, exclude_sock=None):
        to_remove = []
        for sock, nick in list(self.clients):
            if sock is exclude_sock:
                continue
            try:
                sock.send(msg)
            except Exception as e:
                print('Error sending to', nick, e)
                to_remove.append((sock, nick))
        for item in to_remove:
            self.remove_client(item[0])

    def remove_client(self, sock):
        for s, n in list(self.clients):
            if s is sock:
                self.clients.remove((s, n))
                try:
                    s.close()
                except:
                    pass
                print(f'Removed {n}')
                self.broadcast(f'👋 {n} left the chat.'.encode('utf-8'))
                break

    def handle_client(self, client_sock, addr):
        try:
            client_sock.send('NICKNAME'.encode('utf-8'))
            nickname = client_sock.recv(1024).decode('utf-8').strip()
        except Exception:
            client_sock.close()
            return
        self.clients.append((client_sock, nickname))
        print(nickname, 'joined from', addr)
        self.broadcast(f'👋 {nickname} joined the chat!'.encode('utf-8'))
        while True:
            try:
                data = client_sock.recv(1024)
                if not data:
                    self.remove_client(client_sock)
                    break
                # broadcast message to others
                self.broadcast(data, exclude_sock=client_sock)
            except Exception as e:
                print('Client error', e)
                self.remove_client(client_sock)
                break

    def start(self):
        self.server.bind((self.host, self.port))
        self.server.listen(5)
        print(f'🚀 TinyChatServer listening on {self.host}:{self.port}')
        try:
            while True:
                client_sock, addr = self.server.accept()
                t = threading.Thread(target=self.handle_client, args=(client_sock, addr), daemon=True)
                t.start()
        except KeyboardInterrupt:
            print('\nShutting down')
            self.server.close()

if __name__ == '__main__':
    # Warning: running this inside some notebook kernels will block; prefer a terminal
    srv = TinyChatServer(host='127.0.0.1', port=5555)
    print('To test: open another terminal and run a client to connect to 127.0.0.1:5555')
    try:
        srv.start()
    except Exception as e:
        print('Server error', e)
        sys.exit(1)


## 🧪 Minimal Client (runnable)
Use this client from another terminal or notebook to connect to the server. Replace `HOST` with the server IP if running across devices.


In [ ]:
import socket
import threading

def run_client(host='127.0.0.1', port=5555):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((host, port))
    # Wait for nickname prompt
    data = sock.recv(1024).decode('utf-8')
    if data == 'NICKNAME':
        nick = input('Choose a nickname: ').strip()
        sock.send(nick.encode('utf-8'))

    def listen():
        while True:
            try:
                m = sock.recv(1024)
                if not m:
                    print('\nDisconnected from server')
                    break
                print('\n' + m.decode('utf-8'))
            except Exception as e:
                print('Listen error', e)
                break

    t = threading.Thread(target=listen, daemon=True)
    t.start()
    print('Connected! Type messages and press Enter to send. Ctrl+C to quit.')
    try:
        while True:
            msg = input()
            if msg:
                sock.send(f'{nick}: {msg}'.encode('utf-8'))
    except KeyboardInterrupt:
        print('\nClosing')
        sock.close()

# To run: uncomment the line below and run this cell in a new terminal / notebook
# run_client('127.0.0.1', 5555)


## 🔍 How this maps to the GitHub project
- The repo likely contains a server and a client similar to the TinyChatServer and `run_client()` above.
- The notebook version focuses on clarity and safe defaults (127.0.0.1), while the repo targets hotspots and real devices (use the hotspot device's IP).
- For cross‑device tests, find your hotspot device IP (e.g., `192.168.43.1`) and use that as `host` when starting the server and when clients connect.


## 🛠 Tips for running across phones
- Start the hotspot on one phone (this becomes the network). On that phone run the server (using its hotspot IP). On other phones, connect to the hotspot and run client script pointing to the server IP.
- If you can't run Python on the phone, run the client on a laptop/PC connected to the hotspot.
- Use `ipconfig` / `ifconfig` / phone settings to discover the hotspot IP.
- If connections fail: check firewall, ensure server is bound to the hotspot interface (use `0.0.0.0` to accept all interfaces), and use correct IP and port.


## ✅ Next steps & ideas to extend
- Add a GUI (Tkinter, Kivy) for nicer chat UI on phones and desktops
- Use UDP broadcast for simpler LAN discovery (but less reliable)
- Add message timestamps and message history stored in a file
- Secure the chat: add simple authentication, or encrypt messages with TLS

----
If you want, I can now save this notebook file and give you a download link. (I already saved it for you.)